In [3]:
# import packages
import pandas as pd, numpy as np
import json
# import files
import data as dt

1. Replicar 100 bps? -> **10 Basis points: dejar como parametro del backtest**
2. Determinar origin vs destination exchange -> **Kraken es origin, Bitfinex es destination**
3. Como replicar los niveles del origin en el destination (volumen)
4. Validar el método para determinar maker/taker fee: iterativo vs estático
5. Orden de llenado por tiempo de entrada.
6. Evolución de Orderbooks tipo bootstrap.

In [4]:
# load data
ob_krak, ob_bit = dt.read_jsonOB(file_name = 'orderbooks_05jul21.json')


In [4]:
type(ob_krak)

dict

# Alineación de timestamps

In [9]:
list(ob_krak.keys())[0], list(ob_bit.keys())[0], list(ob_bit.keys())[1], list(ob_bit.keys())[2], list(ob_bit.keys())[3]

('2021-07-05T13:06:49.495Z',
 '2021-07-05T13:06:51.077Z',
 '2021-07-05T13:06:52.426Z',
 '2021-07-05T13:06:53.921Z',
 '2021-07-05T13:06:55.563Z')

In [10]:
# filtro  para que destination sea despues de origin
aux = sum(pd.to_datetime(pd.Series((ob_bit.keys()))) < pd.to_datetime(pd.Series(ob_krak.keys()))[0])
[ob_bit.pop(key) for key in list(ob_bit.keys())[:aux]]

[]

In [12]:
10/10000

0.001

# Origin exchange (Kraken)

In [7]:
# OB 1
df_krak = ob_krak[list(ob_krak.keys())[0]]
# midprice
mid_krak = (df_krak['bid'][0]+df_krak['ask'][0])/2
display(df_krak.head(3))
mid_krak

,bid_size,bid,ask,ask_size
0,0.880,28298.3,28298.4,3.352
1,0.350,28293.9,28300.0,0.020
2,0.377,28293.5,28305.4,0.074


28298.35

In [8]:
upper_krak = mid_krak*1.001
lower_krak = mid_krak*0.999
(lower_krak, upper_krak)

(28270.051649999998, 28326.648349999996)

In [9]:
bids_krak = df_krak[df_krak['bid']>lower_krak][['bid','bid_size']]
asks_krak = df_krak[df_krak['ask']<upper_krak][['ask','ask_size']]

In [10]:
[d.shape for d in [df_krak, bids_krak, asks_krak]]

[(100, 4), (30, 2), (23, 2)]

In [11]:
levels = pd.concat([bids_krak, asks_krak], axis=1)

In [12]:
levels

,bid,bid_size,ask,ask_size
0,28298.3,0.880,28298.4,3.352
1,28293.9,0.350,28300.0,0.020
2,28293.5,0.377,28305.4,0.074
3,28292.2,0.011,28305.5,0.123
4,28292.0,1.221,28307.3,0.204
5,28291.2,0.745,28308.2,0.005
6,28291.0,0.396,28310.0,0.004
7,28290.9,0.340,28311.8,0.745
8,28290.8,0.006,28314.9,0.250
9,28290.0,0.003,28315.2,5.000


# Destination exchange (Bitfinex)

In [13]:
# OB 1
df_bit = ob_bit[list(ob_bit.keys())[0]]
# midprice
mid_bit = (df_bit['bid'][0]+df_bit['ask'][0])/2
display(df_bit.head(3))
mid_bit

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


28276.5

# Preliminary approach using loops

In [109]:
# destination exchange
bit_ask = df_bit[['ask','ask_size']]
bit_bid = df_bit[['bid_size','bid']]

bit_ask['ask_added_vol'] = np.zeros(bit_ask.shape[0])
bit_bid['bid_added_vol'] = np.zeros(bit_bid.shape[0])

# origin exchnage
krak_bid = df_krak[['bid_size','bid']]
krak_ask = df_krak[['ask','ask_size']]

<ipython-input-109-3c8ae2bf6895>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bit_ask['ask_added_vol'] = np.zeros(bit_ask.shape[0])
<ipython-input-109-3c8ae2bf6895>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bit_bid['bid_added_vol'] = np.zeros(bit_bid.shape[0])


In [110]:
bid_levels = levels[['bid_size','bid']]
bid_levels['bid_flag'] = ['bid']*bid_levels.shape[0]
bid_levels.rename(columns={'bid_size':'size',
                          'bid':'price',
                          'bid_flag':'type'}, inplace=True)

ask_levels = levels[['ask_size','ask']]
ask_levels['ask_flag'] = ['ask']*ask_levels.shape[0]
ask_levels.rename(columns={'ask_size':'size',
                          'ask':'price',
                          'ask_flag':'type'}, inplace=True)

levels_added = bid_levels.append(ask_levels, ignore_index=True).dropna()

levels_added['transaction_time'] = np.random.uniform(1250, size=levels_added.shape[0])
levels_added.sort_values(by='transaction_time', inplace=True, ignore_index=True)

<ipython-input-110-56ab3a99147f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bid_levels['bid_flag'] = ['bid']*bid_levels.shape[0]
C:\Users\if724\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-110-56ab3a99147f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

### Notas de asesoría

* Añadir latencia de código para simular el tiempo transcurrido en la operación (VARIABLE)
* Si el tiempo total de latencia para llenar todas las ordenes es mayor a el tiempo entre OB update entonces algunas ordenes no se llenan
* El hedge en el origin no es posible realizarlo al instante, existe latencia en procesamiento de transacción.
* Spread en origin se puede reducir introduciendo sólo una fracción del volumen del origin en el destination.
* Replicar niveles en del origin con el inventario en ambos exchanges como restricción
* SLA: 10 bps a partir del Origin ToB y 'x' cantidad de volumen mínimo
* Insights
* **Especificar supuestos**

In [111]:
for i in range(levels_added.shape[0]): # loop to add levels unto destination

    to_fill = levels_added.iloc[0,:]
    
    if to_fill['type']=='bid':
        # identify order's fee structure
        fee_type = 'taker' if to_fill['price'] >= bit_ask.iloc[0]['ask'] else 'maker'


        if fee_type=='taker':

            bit_ask['accum_size'] = bit_ask['ask_size'].cumsum()


            vol_to_fill=to_fill['size']

            bit_ask['remaining_vol'] = vol_to_fill - bit_ask['accum_size']
            to_drop = bit_ask[bit_ask['remaining_vol']>0]

            bit_ask.drop(to_drop.index, inplace=True)
            bit_ask.reset_index(drop=True, inplace=True)

            new_vol = -bit_ask.iloc[0,-1]  # remaing volume on surviving level

            bit_ask.iloc[0,1] = new_vol
            bit_ask.drop(['accum_size', 'remaining_vol'], axis=1, inplace=True) 
            bit_ask.reset_index(drop=True, inplace=True)
            
            levels_added.drop(0, inplace=True)
            levels_added.reset_index(drop=True, inplace=True)
            
            # Hedge transaction in orgin exchange (Kraken)
            
            krak_bid['accum_size'] = krak_bid['bid_size'].cumsum()
            krak_bid['remaining_vol'] = vol_to_fill - krak_bid['accum_size']
            to_drop = krak_bid[krak_bid['remaining_vol']>0]
            
            krak_bid.drop(to_drop.index, inplace=True)
            krak_bid.reset_index(drop=True, inplace=True)
            
            new_vol = -krak_bid.iloc[0,-1] # remaing volume on surviving level
            
            krak_bid.iloc[0,0] = new_vol
            krak_bid.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
            krak_bid.reset_index(drop=True, inplace=True)
            
        else:
            
            bit_bid.rename(columns={'bid_size':'size','bid':'price'}, inplace=True)
            
            to_fill = pd.DataFrame(to_fill).transpose()
            to_fill['bid_added_vol'] = to_fill['size'].values
            
            bit_bid = bit_bid.merge(to_fill[['size','price','bid_added_vol']], 
                                    on=['size','price','bid_added_vol'], how='outer'
                                   ).sort_values(by='price', ascending=False, ignore_index=True) 
        
            bit_bid.rename(columns={'size':'bid_size','price':'bid'}, inplace=True)
            bit_bid = bit_bid.groupby('bid', as_index=False, sort=False).sum()
            bit_bid = bit_bid.reindex(columns=['bid_added_vol', 'bid_size', 'bid'])


            levels_added.drop(0, inplace=True)
            levels_added.reset_index(drop=True, inplace=True)
            
    else:
        
        # identify order's fee structure
        fee_type = 'taker' if to_fill['price'] < bit_bid.iloc[0]['bid'] else 'maker'            
        
        if fee_type == 'taker':
            
            bit_bid['accum_size'] = bit_bid['bid_size'].cumsum()


            vol_to_fill = to_fill['size']

            bit_bid['remaining_vol'] = vol_to_fill - bit_bid['accum_size']
            to_drop = bit_bid[bit_bid['remaining_vol']>0]

            bit_bid.drop(to_drop.index, inplace=True)
            bit_bid.reset_index(drop=True, inplace=True)

            new_vol = -bit_bid.iloc[0,-1] # remaing volume on surviving level

            bit_bid.iloc[0,0] = new_vol
            bit_bid.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
            bit_bid.reset_index(drop=True, inplace=True)
            
            levels_added.drop(0, inplace=True)
            levels_added.reset_index(drop=True, inplace=True)
            
            # Hedge transaction in origin market (Kraken)
            krak_ask['accum_size'] = krak_ask['bid_size'].cumsum()
            krak_ask['remaining_vol'] = vol_to_fill - krak_ask['accum_size']
            to_drop = krak_ask[krak_ask['remaining_vol']>0]
            
            krak_ask.drop(to_drop.index, inplace=True)
            krak_ask.reset_index(drop=True, inplace=True)
            
            new_vol = -krak_ask.iloc[0,-1] # remaing volume on surviving level
            
            krak_ask.iloc[0,1] = new_vol
            krak_ask.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
            krak_ask.reset_index(drop=True, inplace=True)
            
            
        else:
            bit_ask.rename(columns={'ask_size':'size','ask':'price'}, inplace=True)
            
            to_fill = pd.DataFrame(to_fill).transpose()
            to_fill['ask_added_vol'] = to_fill['size'].values
            
            bit_ask = bit_ask.merge(to_fill[['size','price','ask_added_vol']], 
                                    on=['size','price','ask_added_vol'], how='outer'
                                   ).sort_values(by='price', ascending=True, ignore_index=True)               
            bit_ask.rename(columns={'size':'ask_size','price':'ask'}, inplace=True)
            bit_ask = bit_ask.groupby('ask', as_index=False, sort=False).sum()
            bit_ask = bit_ask.reindex(columns=['ask', 'ask_size', 'ask_added_vol'])


            levels_added.drop(0, inplace=True)
            levels_added.reset_index(drop=True, inplace=True)
            


<ipython-input-111-42126608dd32>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  krak_bid['accum_size'] = krak_bid['bid_size'].cumsum()
C:\Users\if724\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [112]:
bit_ob = pd.concat([bit_bid, bit_ask], axis=1)

In [113]:
bit_ob

,bid_added_vol,bid_size,bid,ask,ask_size,ask_added_vol
0,0.798,0.798000,28285.3,28289.0,2.732301,0.000
1,0.326,0.326000,28285.2,28290.0,1.595676,0.000
2,0.300,0.300000,28283.6,28291.0,0.263632,0.000
3,0.250,0.250000,28282.1,28292.0,4.135413,0.000
4,0.018,0.018000,28281.7,28293.0,0.867221,0.000
5,0.093,0.093000,28281.0,28294.0,2.935248,0.000
6,1.500,1.500000,28280.7,28295.0,1.066304,0.000
7,0.177,0.177000,28280.3,28296.0,5.424052,0.000
8,0.300,0.300000,28280.2,28297.0,0.110386,0.000
9,1.491,1.491000,28279.4,28298.0,2.192474,0.000


In [120]:
new_top = (ob_bit[list(ob_bit.keys())[1]].loc['0','bid'], ob_bit[list(ob_bit.keys())[1]].loc['0','ask'])

In [125]:
bids_to_drop = bit_ob[bit_ob['bid']>new_top[0]]
asks_to_drop = bit_ob[bit_ob['ask']<new_top[1]]

In [129]:
bit_ob.drop(bids_to_drop.index, inplace=True)
bit_ob.reset_index(drop=True, inplace=True)

In [131]:
ob_bit[list(ob_bit.keys())[1]]

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000
3,0.003000,28268.0,28281.0,0.050000
4,1.581528,28266.0,28282.0,0.323781
5,2.000887,28265.0,28283.0,1.580312
6,1.309362,28264.0,28284.0,0.353892
7,0.133730,28263.0,28285.0,1.053899
8,0.596041,28262.0,28286.0,0.950994
9,0.188323,28260.0,28287.0,0.157200


In [130]:
bit_ob

,bid_added_vol,bid_size,bid,ask,ask_size,ask_added_vol
0,0.000,14.605659,28275.0,28300.0,0.822599,0.020
1,0.001,0.001000,28274.7,28301.0,0.203518,0.000
2,1.732,1.732000,28274.3,28302.0,2.916168,0.000
3,0.100,0.100000,28273.4,28305.4,0.074000,0.074
4,0.000,0.005124,28273.0,28305.5,0.123000,0.123
5,1.490,1.490000,28272.6,28307.3,0.204000,0.204
6,0.000,0.010158,28272.0,28308.2,0.005000,0.005
7,0.002,0.002000,28270.3,28310.0,0.004000,0.004
8,0.000,0.003000,28268.0,28311.8,0.745000,0.745
9,0.000,1.581528,28266.0,28314.9,0.250000,0.250


In [121]:
new_top

(28275.0, 28278.0)

In [27]:
krak_ask.reset_index(drop=True, inplace=True)
krak_ob = pd.concat([krak_bid, krak_ask], axis=1)

# Next steps

1. Sacar diferencias en volumen de cada nivel entre el OB output `ob_bit` y el siguiente OB destination.
**Termina ciclo**